In [1]:
from collections import OrderedDict, namedtuple
from time import time

In [2]:
Order = namedtuple("Order", ["price", "qty", "side"])

class Book:
    def __init__(self):
        self.orderedBidsBook = OrderedDict()
        self.orderedOffersBook = OrderedDict()
        self.orders = {}

    def __setitem__(self, id, args):
        """
        Args are price, qty, side
        """
        original_order = self.orders[id] if id in self.orders else None
        new_order = Order(*args)
        
        if original_order:
            # this is an update            
            # check if we can just change order or need to move it
            if self._keep_position(original_order, new_order):
                old_orders = self._get_orders(original_order.side, original_order.price)
                old_orders[id] = new_order
            else:
                del self[id]
                new_orders = self._get_orders(new_order.side, new_order.price)
                new_orders[id] = new_order
        else:
            new_orders = self._get_orders(new_order.side, new_order.price)
            new_orders[id] = new_order
        
        self.orders[id] = new_order
    
    def __delitem__(self, id):
        """
        Args are price, qty, side
        """
        original_order = self.orders[id] if id in self.orders else None
        if original_order:
            del self.orders[id]
            old_orders = self._get_orders(original_order.side, original_order.price)
            
            del old_orders[id]
            if len(old_orders) == 0:
                cache = self.orderedBidsBook if original_order.side == "BID" else self.orderedOffersBook
                del cache[original_order.price]
                
    def __getitem__(self, id):
        return self.orders[id]
            

        
    def __iter__(self):
        for vals in self.orderedOffersBook.values():
            for item in vals.items():
                yield item
            
        for vals in self.orderedBidsBook.values():
            for item in vals.items():
                yield item
    
    def _get_orders(self, side, price):
        cache = self.orderedBidsBook if side == "BID" else self.orderedOffersBook
        if price in cache:
            return cache[price]
        else:
            orders = {}
            cache[price] = orders
            return orders
        
    def _keep_position(self, original_order, new_order):
        if original_order.side == new_order.side and original_order.price == new_order.price:
            return True
        return False
    
    def __str__(self):
        return f"{self.orderedOffersBook}, {self.orderedBidsBook}"

In [3]:
book = Book()
book[1000001] = 100, 0.5, "BID"
book[1000002] = 99, 0.5, "BID"
book[1000003] = 101, 0.5, "BID"

book[1000004] = 103, 0.5, "ASK"
book[1000005] = 104, 0.5, "ASK"
book[1000006] = 102, 0.5, "ASK"

print(book)

print(book[1000003])
print(book[1000006])

book[1000001] = 101, 0.5, "BID"
book[1000002] = 100, 0.5, "BID"
book[1000003] = 101, 0.9, "BID"

book[1000004] = 102, 0.5, "ASK"
book[1000005] = 103, 0.5, "ASK"
book[1000006] = 102, 0.9, "ASK"

print(book)

print("*****Ordered*****")
for item in book:
    print(item)
    
    
del book[1000001]
del book[1000003]

print(book)

OrderedDict([(103, {1000004: Order(price=103, qty=0.5, side='ASK')}), (104, {1000005: Order(price=104, qty=0.5, side='ASK')}), (102, {1000006: Order(price=102, qty=0.5, side='ASK')})]), OrderedDict([(100, {1000001: Order(price=100, qty=0.5, side='BID')}), (99, {1000002: Order(price=99, qty=0.5, side='BID')}), (101, {1000003: Order(price=101, qty=0.5, side='BID')})])
Order(price=101, qty=0.5, side='BID')
Order(price=102, qty=0.5, side='ASK')
OrderedDict([(102, {1000006: Order(price=102, qty=0.9, side='ASK'), 1000004: Order(price=102, qty=0.5, side='ASK')}), (103, {1000005: Order(price=103, qty=0.5, side='ASK')})]), OrderedDict([(101, {1000003: Order(price=101, qty=0.9, side='BID'), 1000001: Order(price=101, qty=0.5, side='BID')}), (100, {1000002: Order(price=100, qty=0.5, side='BID')})])
*****Ordered*****
(1000006, Order(price=102, qty=0.9, side='ASK'))
(1000004, Order(price=102, qty=0.5, side='ASK'))
(1000005, Order(price=103, qty=0.5, side='ASK'))
(1000003, Order(price=101, qty=0.9, s